In [1]:
!pip install TTS torch torchaudio SpeechRecognition pydub huggingface_hub

# Conversão de extensão

In [2]:
from pydub import AudioSegment

# Carregar arquivo mp3 e converter para wav
audio = AudioSegment.from_mp3("erika_hilton.mp3")
audio.export("erika_hilton.wav", format="wav")

<_io.BufferedRandom name='erika_hilton.wav'>

# Pré-processamento

In [4]:
import os
import pandas as pd
import speech_recognition as sr
from pydub import AudioSegment

# Instalações necessárias no Google Colab
!pip install SpeechRecognition pydub
!apt-get install ffmpeg  # Para manipulação de áudio

# Função para transcrever áudio
def transcribe_audio(audio_path):
    recognizer = sr.Recognizer()

    # Converter o arquivo de áudio para WAV se necessário
    if audio_path.endswith(".mp3"):
        audio = AudioSegment.from_mp3(audio_path)
        audio_path = audio_path.replace(".mp3", ".wav")
        audio.export(audio_path, format="wav")

    # Transcrição
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)  # Lê o arquivo de áudio
        try:
            # Usando o Google Web Speech API para transcrição em pt-BR
            transcription = recognizer.recognize_google(audio_data, language='pt-BR')
            return transcription
        except sr.UnknownValueError:
            print(f"Não foi possível transcrever o áudio: {audio_path} (não reconhecido)")
            return None
        except sr.RequestError as e:
            print(f"Erro na requisição: {e}")
            return None

# Diretório onde os arquivos de áudio estão armazenados
audio_folder = '/content/audio_files/'  # Crie uma pasta chamada audio_files em /content/ para armazenar seus áudios

# Verifica se o diretório de áudio existe
os.makedirs(audio_folder, exist_ok=True)

# Lista para armazenar os dados
data = []

# Percorrer os arquivos de áudio e transcrever
for file_name in os.listdir(audio_folder):
    if file_name.endswith('.wav') or file_name.endswith('.mp3'):
        audio_path = os.path.join(audio_folder, file_name)
        transcription = transcribe_audio(audio_path)
        if transcription:
            data.append({"file_name": file_name, "transcription": transcription})
        else:
            print(f"Transcrição falhou para o arquivo: {file_name}")

# Verifica se há dados para salvar
if data:
    # Criar DataFrame e salvar como CSV
    df = pd.DataFrame(data)
    csv_file_path = '/content/transcriptions.csv'  # Salvar o CSV em /content/
    df.to_csv(csv_file_path, index=False, header=["file_name", "transcription"])
    print("Transcrições salvas em:", csv_file_path)
else:
    print("Nenhuma transcrição foi realizada. Verifique os arquivos de áudio.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Transcrições salvas em: /content/transcriptions.csv


# Treinamento Modelo de Clonagem de Voz

Login Hugging Face

In [6]:
import os
import pandas as pd
from TTS.api import TTS
from huggingface_hub import login

# Conecte-se ao Hugging Face para acessar o modelo
login("YOUR_HF_TOKEN")  # Substitua "YOUR_HF_TOKEN" pelo seu token do Hugging Face.

# Caminho para os dados
audio_folder = '/content/audio_files/'
csv_file = '/content/transcriptions.csv'

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
!pip install -U mpmath

In [2]:
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=False)

# generate speech by cloning a voice using default settings
tts.tts_to_file(text="Levei muito tempo para desenvolver uma voz e agora que a tenho não vou ficar em silêncio.",
                file_path="output.wav",
                speaker_wav="/content/erika_hilton.wav",
                language="pt")

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Text splitted to sentences.
['Levei muito tempo para desenvolver uma voz e agora que a tenho não vou ficar em silêncio.']


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 > Processing time: 50.29472756385803
 > Real-time factor: 7.171486955400088


'output.wav'

In [3]:
from IPython.display import Audio

Audio("output.wav")